# Imports

In [1]:
!git config --global core.autocrlf true

In [2]:
# könyvtárak beimportálása
import pandas as pd
import os
import SimpleITK as sitk
import numpy as np
import imageio
import matplotlib.pyplot as plt
import seaborn as sb
import pydicom
import warnings
import cv2
from itertools import islice
import random
import glob

In [3]:
#táblázatos adatok beolvasása

readIn = pd.read_csv("data/scd_patientdata.csv")
df_patientdata = pd.DataFrame(readIn)
pd.set_option("display.max_columns", None)
print(df_patientdata.shape)

(45, 5)


In [4]:
df_patientdata.head(50)

,PatientID,OriginalID,Gender,Age,Pathology
0,SCD0000101,SC-HF-I-1,Male,53,Heart failure with infarct
1,SCD0000201,SC-HF-I-2,Male,48,Heart failure with infarct
2,SCD0000301,SC-HF-I-4,Male,79,Heart failure with infarct
3,SCD0000401,SC-HF-I-5,Male,45,Heart failure with infarct
4,SCD0000501,SC-HF-I-6,Male,60,Heart failure with infarct
5,SCD0000601,SC-HF-I-7,Male,74,Heart failure with infarct
6,SCD0000701,SC-HF-I-8,Male,46,Heart failure with infarct
7,SCD0000801,SC-HF-I-9,Male,57,Heart failure with infarct
8,SCD0000901,SC-HF-I-10,Male,69,Heart failure with infarct
9,SCD0001001,SC-HF-I-11,Male,55,Heart failure with infarct


In [5]:
#képeket tartalmazó képek útvonalának elmentése dictionary-be, 
#ahol a key azok az egyes mappákat jelöli, 
#a value meg egy array a benne található fájlokkal

script_dir = os.path.dirname(os.path.abspath('Cardiac MRI segmentation.ipynb'))

data_folder = os.path.join(script_dir, 'data', 'SCD_IMAGES')

files = {}

for dirname, _, filenames in os.walk(data_folder):
    dirname = dirname.replace('\\', '/')

    if dirname not in files:
        file_paths = [os.path.join(dirname, filename) for filename in filenames]
        files[dirname] = file_paths

In [6]:
#a dictionaryt átalakítom egy olyan listává, ahol minden elem, egy fájlt jelöl és aztán randomizálom a sorrendet a tanításhoz

files_array=[]
for folder in files.keys():
    for file in files[folder]:
        if (file.split('.')[1]=='dcm'):
            files_array.append(file)

np.random.shuffle(files_array)

In [7]:
len(files_array)

48475

# Image preprocessing

In [8]:
#egy darab dicom fájl betöltése

def load(file):
    dicom_data=pydicom.dcmread(file)
    return dicom_data

### Format check
The only data thats was pulled into the dataframe is DICOM so it's already done

### Sizing

In [9]:
# egy darab np.array-ben tárolt kép átméretezése

def resize(data):
    resized_image = cv2.resize(data.pixel_array, (256, 256))
    return [data.PatientID,resized_image]

### Normalizing data

In [10]:
#egy darab np.array-ben tárolt kép pixeleinek normalizálása

def normalize(item):
    pixel = item[1]
    minimum = pixel.min()
    maximum = pixel.max()

    if minimum < 0:
        pixel = pixel + abs(minimum)
        maximum += abs(minimum)
        
    if maximum != 0:
        pixel = pixel / maximum
    return [item[0], pixel]

### Augmentation

If needed. For now, it's empty.

## Preprocess

In [11]:
type(files)

dict

In [12]:
files_len=len(files_array)

In [13]:
# betölt egy batchnyi képet, elvégzi a preprocessinget, majd elmenti a megadott fájlba

def convert_and_append_to_npz(dicom_files_batch, npz_file, start_index, files_len):
    image_array=[]
    
    for i, dicom_file in enumerate(dicom_files_batch):
        print(f'Loading:     {start_index + i}/{files_len}', end='\r')
    
        loaded_files = load(dicom_file)
        
        print(f'Resizing:    {start_index + i}/{files_len}', end='\r')
        sized_data = resize(loaded_files)
        print(f'Normalizing: {start_index + i}/{files_len}', end='\r')
        normalized_data = normalize(sized_data)
        image_array.append((normalized_data[0],normalized_data[1]))
    new_ids = np.array([item[0] for item in image_array])
    new_images = np.array([item[1] for item in image_array])
    

    print(f'Saving: {start_index} to {start_index + len(dicom_files_batch) - 1}       ', end='\r')
    np.savez(npz_file, ids=new_ids, images=new_images)


In [14]:
#megnézi, hogy vannak-e olyan fájlok, mint aminek elmentené az új, előkészített képeinket
#ha van, törli őket
#utána szétbontja az fájlnév listát 12 részre és preprocesszálja őket

split_num=12
base_name = 'data/images'

for j in range(split_num):
    file_name = f'{base_name}_split{j}.npz'
    
    if os.path.exists(file_name):
        os.remove(file_name)


total_files = len(files_array)
split_size = total_files // split_num

for j in range(split_num):
    split_name = f'{base_name}_split{j}.npz'

    start_index = j * split_size
    end_index = (j + 1) * split_size if j < split_num-1 else total_files

    split_files = list(files_array)[start_index:end_index]

    convert_and_append_to_npz(split_files, split_name, start_index, files_len)

    print(f"{j+1}/{split_num} split is done           ")

1/12 split is done           
2/12 split is done           
3/12 split is done           
4/12 split is done           
5/12 split is done           
6/12 split is done           
7/12 split is done           
8/12 split is done           
9/12 split is done           
10/12 split is done           
11/12 split is done           
12/12 split is done           


# Merging the data

# Defining input and output


A következőben a táblázatos adatok előkészítését végzem el. Az gender egy flag változó lesz, a korból pedig egy korcsoportos dummy változó. A diagnózis label-lé lesz alakítva

### Is_Male
Flag változó

In [15]:
df_patientdata['Is_Male'] = df_patientdata['Gender'].apply(lambda x: 1 if x == 'Male' else 0)

### Pathology
Dummy változóvá alakítás

| Szöveges Érték                       | Címke | 
|--------------------------------------|-------|
| Normal                               | 0     | |
| Heart failure with infarct           | 1    |  |
| Heart failure without infarct        | 2   |   |
| Hypertrophy                          | 3  |    |


In [16]:
dict = {'Normal': 0, 'Heart failure with infarct': 1, 'Heart failure without infarct' :2, 'Hypertrophy': 3}

In [17]:
df_patientdata['Pathology']=df_patientdata['Pathology'].map(dict)

### Age Group: 
10 évenkénti csoportokban vannak, így dummy változóként lehet használni. Nem tudhatjuk hogy a kor lineárisan befolyásolja-e a betegségek kialakulását.

In [18]:
df_patientdata['Age_Group'] = (df_patientdata['Age']/10).astype(int)

In [19]:
ps_dummies = pd.get_dummies(df_patientdata['Age_Group'], prefix = 'Age_Group', drop_first = True)
df_patientdata = df_patientdata.merge(ps_dummies, left_index = True, right_index = True, how = 'left')

In [20]:
df_patientdata=df_patientdata.drop(['Gender', 'Age', 'Age_Group', 'OriginalID'], axis=1)

In [21]:
#átalakított csv mentése

df_patientdata.to_csv('data/df_patientdata.csv', index=False)


In [22]:
del df_patientdata